# AsyncIO in Rust play ground

see https://blog.logrocket.com/a-practical-guide-to-async-in-rust/

## Learnings

* **async functions** always return **futures**
* futures await their completions, and are pending as long as they are not available
* **async functions** are **coroutines** (stop at await, continue after await)
* **async functions** require a **runtime**, e.g. tokyo

* 'todo!' macro panics and is part of stdlib


In [2]:
:sccache 1
:dep futures = "0.3.*"
:dep tokio = {version = "0.3.3", features = ["full"] }
:dep env_logger = "0.8.3"
:dep log = "0.4.*"
:dep chrono = "0.4.19"

sccache: true


In [3]:
use std::io::Write;
use chrono::Local;
use env_logger::Builder;
use log::LevelFilter;

fn config_logger() {
    Builder::new()
        .format(|buf, record| {
            writeln!(buf,
                "{} [{}] - {}",
                Local::now().format("%H:%M:%S%.3f"),
                record.level(),
                record.args()
            )
        })
        .filter(None, LevelFilter::Debug)
        .init();
}

config_logger();
log::debug!("logger configured");

21:03:28.958 [DEBUG] - logger configured


## Start a runtime explicitely

In [4]:
use futures::prelude::*;
use tokio::prelude::*;
config_logger();

async fn app() {
    log::info!("async done with explicite runtime call");
}

fn main() {
    let mut rt = tokio::runtime::Runtime::new().unwrap();
    // just instanciate the (async function) app - do not run it
    let future = app();
    log::debug!("App instanciated");
    
    // block until the async function is finished
    rt.block_on(future);
    log::debug!("Unblocked and finished");
}

main()


21:03:41.216 [DEBUG] - App instanciated
21:03:41.216 [INFO] - async done with explicite runtime call
21:03:41.216 [DEBUG] - Unblocked and finished


()

In [5]:
:vars

Variable,Type


## Start a runtime compact form

In [6]:
#[tokio::main]
async fn main() {
    println!("async done in toyo runtime injection");
}

main()

async done in toyo runtime injection


()

## Spawn blocking or CPU-intensive tasks


In [7]:
use tokio::task;
config_logger();

fn fib_cpu_intensive(n: u32) -> u32 {
    match n {
        0 => 0,
        1 => 1,
        n => fib_cpu_intensive(n - 1) + fib_cpu_intensive(n - 2),
    }
}

async fn app() {
    log::info!("async done with explicite runtime call");
}

#[tokio::main]
async fn main() {
    let threadpool_future = task::spawn_blocking(||log::info!("Fibonacci number of 30 is {}", fib_cpu_intensive(30)));
    let concurrent_future = task::spawn(app());
    log::debug!("All initialized - spawn_blocking also starts a thread already.");
    let _ = threadpool_future.await;  // waits until the the long running functions finished in the thread.
    let _ = concurrent_future.await;  // Actually call the async function app
    log::debug!("All finisned i.e. all futures have been completed")
    
}
main()

21:03:57.606 [DEBUG] - All initialized - spawn_blocking also starts a thread already.
21:03:57.606 [INFO] - async done with explicite runtime call
21:03:57.612 [INFO] - Fibonacci number of 30 is 832040
21:03:57.613 [DEBUG] - All finisned i.e. all futures have been completed


()

## Async functions return futures

- await the future and proceed 

In [8]:
use tokio::time::{sleep, Duration};
config_logger();

async fn wait_ms_async(time_in_ms: u64) -> u64 {
    sleep(Duration::from_millis(time_in_ms)).await;
    log::debug!("{} ms have elapsed", time_in_ms);
    time_in_ms
}

async fn request(n: u16) -> Result<u16, ()> {
    log::debug!("Request n: {}", n);
    // future::ok resolves directly to a ready; i.e. a completed future
    // the following await converts the future into an ordinary function
    future::ok::<_,()>(n).await
}

async fn wait_two_times_sequentially() -> u64 {
    let mut sum = 0_u64;
    let f = wait_ms_async(300)
        .then(|first_return_value| {
            // 
            log::debug!("1st result {}", first_return_value);
            sum = first_return_value;
            wait_ms_async(500) });
    f.await;
    sum 
    // 100_u64
}

#[tokio::main]
async fn main() {
    let resp1 = task::spawn(request(2));
    let resp2 = task::spawn(wait_ms_async(150));
    let resp3 = task::spawn(wait_ms_async(200));
    let resp4 = task::spawn(wait_two_times_sequentially());
    log::debug!("All initialized");
    let resp1_result = resp1.await;
    let resp2_result = resp2.await;
    let resp3_result = resp3.await;
    let resp4_result = resp4.await;
    log::info!("Result 2: {} 3: {}", resp2_result.unwrap(), resp4_result.unwrap());
    log::debug!("Finished both");
}
main()

21:04:05.048 [DEBUG] - All initialized
21:04:05.049 [DEBUG] - Request n: 2
21:04:05.203 [DEBUG] - 150 ms have elapsed
21:04:05.251 [DEBUG] - 200 ms have elapsed
21:04:05.353 [DEBUG] - 300 ms have elapsed
21:04:05.353 [DEBUG] - 1st result 300
21:04:05.855 [DEBUG] - 500 ms have elapsed
21:04:05.855 [INFO] - Result 2: 150 3: 300
21:04:05.855 [DEBUG] - Finished both


()

## async Equivalence

async provides syntactic sugger

A function that returns a Future. The future is not part of the return.
see https://medium.com/@alistairisrael/demystifying-closures-futures-and-async-await-in-rust-part-3-async-await-9ed20eede7a4

In [9]:
async fn async_returns_i32() -> i32 {
    42
}

fn returns_future_i32() -> impl Future<Output = i32> {
    future::ready(42)
}

fn returns_async_block_i32() -> impl Future<Output = i32> {
    async { 42 }
}

## await Equivalence

await is like waiting until the future completes, i.e. is not pending anymore.

In [10]:
    let x = 42;
    let async_capture = future::lazy(|_| {
        debug!("in async_capture, x => {}", x);
    });
    rt.block_on(async_capture);

Error: Sorry, the type futures::future::Lazy<[closure@src/lib.rs:192:38: 194:6]> cannot currently be persisted

In [11]:
config_logger();

fn returns_impl_future_i32() -> impl Future<Output = i32> {
    future::ready(42)
}

let mut rt = tokio::runtime::Runtime::new().unwrap();
rt.block_on(future::lazy(|_| debug!("in rt.block_on()")));
let r0 = rt.block_on(future::ready("Hello from rt.block_on()"));
log::debug!("{}", r0);

let r1 = rt.block_on(returns_impl_future_i32());
log::debug!("returns_impl_future_i32() -> {}", r1);

// do not run - this one loops forever
# force compile error


Error: expected one of `!` or `[`, found `force`

Error: cannot find macro `debug` in this scope

Error: cannot find macro `debug` in this scope

In [15]:
use log::debug;

fn returns_impl_future_i32() -> impl Future<Output = i32> {
    future::ready(42)
}

let mut rt = tokio::runtime::Runtime::new().unwrap();
rt.block_on(
    async {
        debug!("in rt.block_on()");
        let r0 = future::ready("Hello from rt.block_on()").await;
        debug!("{}", r0);
        let r1 = returns_impl_future_i32().await;
        debug!("returns_impl_future_i32() -> {}", r1);
    }
)

Error: expected one of `::`, `;`, or `as`, found `:`